<a href="https://colab.research.google.com/github/tig1795/word_embeddings_Lexika/blob/main/Bert_Wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [3]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 9.8 MB/s 
     |████████████████████████████████| 596 kB 54.4 MB/s 
     |████████████████████████████████| 3.3 MB 56.1 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 895 kB 65.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch
from transformers import AutoModel, AutoTokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-german-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474k [00:00<?, ?B/s]

In [6]:
text = "schwul"
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

['[CLS]', 'schw', '##ul', '[SEP]']


In [7]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

In [8]:
indexed_tokens

[3, 1789, 236, 4]

In [9]:
segments_ids = [1] * len(tokenized_text)

In [10]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = AutoModel.from_pretrained('bert-base-german-cased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [11]:
tokens_tensor

tensor([[   3, 1789,  236,    4]])

In [12]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [13]:
encoded_layers

'last_hidden_state'

bspwort: "schwul"

1. bert model laden ✅
2. embedding von bspwort extrahieren aus dem bert model (featur extraction) ✅
3. irgendwie most similar embeddings finden
4. diese embeddings in tokens umwandeln
5. profit

In [15]:
from transformers import BertTokenizer, BertModel

import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')

model = BertModel.from_pretrained('bert-base-german-cased')

word = "schwul"

inputs = tokenizer(word, return_tensors="pt")

outputs = model(**inputs)

word_vect = outputs.pooler_output.detach().numpy()

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
w1 = model(**tokenizer("bank", return_tensors="pt")).pooler_output.detach().numpy()
w2 = model(**tokenizer("bank", return_tensors="pt")).pooler_output.detach().numpy()

In [17]:
w2.shape

(1, 768)

In [18]:
w1.shape

(1, 768)

In [19]:
from numpy import dot
from numpy.linalg import norm

a = w1
b = w2.T

cos_sim = dot(a, b)/(norm(a)*norm(b))

# 1. das hier in ne funktion packen
# 2. liste von allen wörter aus bert übergeben
# für jedes wort (bzw. subtoken), ähnlichkeit berechnen wir hier in der zelle

In [20]:
cos_sim[0][0]

1.0

In [21]:
import operator
def foo(w, l):
  w1 = model(**tokenizer(w, return_tensors="pt")).pooler_output.detach().numpy()

  output = {}

  for tw in l:
    w2 = model(**tokenizer(tw, return_tensors="pt")).pooler_output.detach().numpy()
    w2 = w2.T
    cos_sim = dot(w1, w2)/(norm(w1)*norm(w2))
    output[tw] = cos_sim[0][0] # schlauer mit np.squeeze

  return sorted(output.items(), key=lambda kv: kv[1], reverse=True)

In [22]:
l = ["lesbisch", "blöd", "nett", "normal"]
w = "schwul" 

In [23]:
foo(w, l)

[('nett', 0.91946226),
 ('normal', 0.9110073),
 ('lesbisch', 0.894619),
 ('blöd', 0.8641924)]

- idee 1: feste liste von wörtern nehmen, die mit bert vergleichen wie oben. dann most_similar bei word2vec machen und gucken, wieviele der top 100 wörter oder so in den top 100 vorkommen
- idee 2: mit word2vec anfangen, top 100 wörter ausgeben. die liste extrahieren und bert als liste übergeben. gucken, ob die verteilung der wörter übereinstimmt, d.h. die top 100 gleiche reihenfolge
- idee 3: alle deutschen wörter nehmen haha: https://www.google.com/search?q=liste+aller+deutschen+w%C3%B6rter&rlz=1C1GCEU_deDE945DE945&oq=liste+aller+deutschen+w%C3%B6rter&aqs=chrome..69i57j0i22i30l2.6242j0j7&sourceid=chrome&ie=UTF-8

## Ergebnis für w = "schwul"

In [ ]:
w = "schwul"

schwul = foo(w, tokenizer.vocab)

In [ ]:
schwul

[('##flugzeug', 0.9480279),
 ('##zös', 0.94353014),
 ('##rankheit', 0.9417904),
 ('##frauen', 0.9407319),
 ('Junge', 0.9404017),
 ('##fraktion', 0.9367636),
 ('streng', 0.936416),
 ('##männer', 0.9363357),
 ('##kretär', 0.93545467),
 ('Süß', 0.9352935),
 ('Schatz', 0.93489677),
 ('##emark', 0.93440336),
 ('##Mann', 0.9338226),
 ('##gläubiger', 0.9337395),
 ('##frau', 0.9334732),
 ('##stiftung', 0.93316907),
 ('##sextrem', 0.93302345),
 ('##geschlecht', 0.9327477),
 ('##Frauen', 0.93273765),
 ('##fechtbar', 0.9326009),
 ('##fahrzeug', 0.93243057),
 ('##blig', 0.9324282),
 ('##ölbe', 0.93179584),
 ('##geschwind', 0.931722),
 ('##rankenhaus', 0.9314185),
 ('##raucht', 0.93140924),
 ('Star', 0.9313961),
 ('##gefü', 0.9312438),
 ('##häuser', 0.93119556),
 ('##Po', 0.931094),
 ('##schrauber', 0.93080664),
 ('Liebhaber', 0.9306864),
 ('wild', 0.9304709),
 ('##ember', 0.9304351),
 ('##pekt', 0.9303036),
 ('##rach', 0.9302786),
 ('Vize', 0.9302252),
 ('##sexamen', 0.93001556),
 ('##ufakt', 0.92

Ergebnis für die ähnlichsten Wörter für schwul unter allen Wörtern des Vokabulars des Bert-Modells. 


 '##rankheit': 0.9417904,
 'Junge': 0.9404016,
 '##männer': 0.93633574,
 'Süß': 0.9352935,
 'Schatz': 0.93489677,
 '##Mann': 0.93382263,
 '##sextrem': 0.9330235,
 '##geschlecht': 0.9327478,
 '##Po': 0.93109405,
 'Liebhaber': 0.93068653,
 'wild': 0.9304709,
 'Krank': 0.9293221,
 'sex': 0.92920583,
 'Liebe': 0.9286042,
 'Queen': 0.9282095,

War so zu erwarten. Allerdings bedenklich, dass eines der ähnlichsten Wörter Krankheit zu sein scheint, bei einem Modell, dass auf aktuellen Wikipedia-Einträgen trainiert wurde!

# Vergleich dazu mit den 50 ähnlichsten Wörtern von Word2Vec 

In [ ]:
s = "schwul"

c = ['homosexuell', 'bisexuell', 'heterosexuell', 'lesbisch', 'Rassist', 'transsexuell', 'benehme', 'Schwuler', 'hasse', 
'paedophil', 'hetero', 'sexsuechtig', 'froehlicher_Mensch', 'Schwuchtel', 'luege', 'erzaehlst', 'anluegen', 'netter_Mensch', 
'cooler_Typ', 'netter_Kerl', 'Weichei', 'Schwulsein', 'aussiehst', 'deppert', 'geoutet', 'gluecklicher_Mensch', 'schwul_lesbisch',
'geradeheraus', 'verarschen', 'schwule', 'schwuler', 'fremdgegangen', 'glaeubiger_Mensch', 'bescheuert', 'echter_Fan', 'Du_wolltest', 'leidtut',
'Kontrollfreak', 'Quotenfrau', 'ausziehe', 'uebelst', 'Hurensohn', 'groessenwahnsinnig', 'Heulsuse', 'Lesbe', 'Arschloecher',
'eingebildet', 'veraeppelt', 'vorlaut', 'rumlaeuft' ]

schwulword2vec = foo(s, c)

In [ ]:
schwulword2vec

[('schwul_lesbisch', 0.96438485),
 ('schwuler', 0.9340011),
 ('Schwuchtel', 0.9327613),
 ('Kontrollfreak', 0.9309159),
 ('cooler_Typ', 0.9264718),
 ('Schwulsein', 0.9259506),
 ('rumlaeuft', 0.9249543),
 ('sexsuechtig', 0.9246818),
 ('Hurensohn', 0.9222037),
 ('netter_Kerl', 0.92082256),
 ('netter_Mensch', 0.92043525),
 ('schwule', 0.9167998),
 ('Weichei', 0.9156029),
 ('Quotenfrau', 0.912502),
 ('gluecklicher_Mensch', 0.9092399),
 ('vorlaut', 0.90738213),
 ('Arschloecher', 0.90168864),
 ('froehlicher_Mensch', 0.90053207),
 ('echter_Fan', 0.897399),
 ('bescheuert', 0.89687157),
 ('lesbisch', 0.894619),
 ('Schwuler', 0.8940697),
 ('glaeubiger_Mensch', 0.89212),
 ('fremdgegangen', 0.89159644),
 ('groessenwahnsinnig', 0.888275),
 ('Lesbe', 0.885837),
 ('verarschen', 0.8832403),
 ('luege', 0.8784347),
 ('eingebildet', 0.8776532),
 ('bisexuell', 0.87702626),
 ('transsexuell', 0.87298214),
 ('leidtut', 0.87161523),
 ('Heulsuse', 0.87160146),
 ('deppert', 0.86547),
 ('geradeheraus', 0.8579069)

In [ ]:
#Test zum Listenvergleich 

for item1 in schwul[1::2]:
    for item2 in schwulword2vec[1::2]:
        if item1 == item2:
            print(item1)

# Versuch mit Feminismus

## Automatisch generierte ähnlichste Wörter zum Wort Feminismus

In [ ]:
w = "Feminismus"

Hi = foo(w, tokenizer.vocab)

In [ ]:
Hi

[('Journalismus', 0.9320039),
 ('Islam', 0.9153972),
 ('Ethik', 0.9138482),
 ('Intellekt', 0.9127552),
 ('Kapitalismus', 0.9097322),
 ('Rassismus', 0.90690005),
 ('Identität', 0.9068035),
 ('Tierschutz', 0.9052345),
 ('Lehre', 0.90509796),
 ('Philosophie', 0.9040371),
 ('Christentum', 0.90383524),
 ('Toleranz', 0.9032742),
 ('Reflex', 0.9026124),
 ('Verhalten', 0.9022104),
 ('##ifizierung', 0.90194094),
 ('Christen', 0.9017803),
 ('Ökonomie', 0.9015981),
 ('Lobby', 0.90008956),
 ('Religion', 0.8995289),
 ('Kontext', 0.8992756),
 ('Meinungs', 0.8991688),
 ('Intervention', 0.89913595),
 ('Standpunkt', 0.8990546),
 ('Haltung', 0.8989758),
 ('Konsens', 0.8988254),
 ('Evolution', 0.8984677),
 ('Glauben', 0.8980897),
 ('Politische', 0.89781255),
 ('sexuelle', 0.89778465),
 ('##izismus', 0.89633685),
 ('Report', 0.89611125),
 ('Analyse', 0.8955748),
 ('Kulturen', 0.8952659),
 ('Werbung', 0.89498216),
 ('Opposition', 0.89415294),
 ('Aspekt', 0.89411),
 ('Meinung', 0.8938122),
 ('Sexual', 0.893

Ergebnis ist nicht gut, die angegebenen ähnlichsten Wörter haben meistens nichts mit Feminismus zu tun.

In [ ]:
len(Hi)

30000

# Versuch mit den 50 ähnlichsten Wörtern aus Word2Vec für das Wort "Feminismus"

In [ ]:
f = "Feminismus"
c = ['feministische', 'sexuelle_Befreiung', 'Frauenbewegung', 'Emanzipation', 'Feministinnen', 'feministischen', 'Rassismus_Sexismus', 'Sexismus', 'Maennerbild', 'Sexualitaet', 'feministischer', 'Fundamentalismus', 
'Geschlechterrollen', 'Antikapitalismus', 'Konservatismus', '68ern', 'Liberalismus', 'Dissidenz', 'antifeministische', 'Alltagssexismus', 'Frauenhass', 'Kapitalismuskritik', 'Frauenemanzipation', 'Veganismus',
'Misogynie', 'Geschlechterpolitik', 'emanzipatorische', 'weibliche_Sexualitaet', 'Political_Correctness', 'Maennerrechtsbewegung', 'Islamismus', 'Sexismus_Rassismus', 'feministisch', 'Paternalismus', 'Kapitalismus',
'Neoliberalismus', 'Anarchismus', 'Pluralismus', 'Alice_Schwarzer', 'Diskurs', 'emanzipatorischen', 'Systemkritik', 'Aktivismus', 'Islam', 'Machismo', 'Frauenfrage', 'Schwulsein', 'Konsumkritik', 'Konsumismus',
'Sexualmoral']

In [ ]:
L = foo(f, c)

In [ ]:
#Ergebnisliste für word-similarity mit Bert

L 

[('Antikapitalismus', 0.95683116),
 ('Emanzipation', 0.9536),
 ('Aktivismus', 0.94995266),
 ('Konsumismus', 0.9478731),
 ('Pluralismus', 0.9426455),
 ('Anarchismus', 0.9423294),
 ('Frauenemanzipation', 0.9410226),
 ('Neoliberalismus', 0.9405782),
 ('feministisch', 0.9402779),
 ('Fundamentalismus', 0.9391826),
 ('Diskurs', 0.93131435),
 ('Konservatismus', 0.93041545),
 ('Sexismus', 0.92430687),
 ('feministische', 0.9228124),
 ('Veganismus', 0.92188174),
 ('Systemkritik', 0.92104954),
 ('Geschlechterpolitik', 0.92042434),
 ('Alltagssexismus', 0.92014796),
 ('Islam', 0.9153972),
 ('Dissidenz', 0.91383886),
 ('Liberalismus', 0.91362774),
 ('Feministinnen', 0.91338545),
 ('Konsumkritik', 0.9115436),
 ('Kapitalismus', 0.9097322),
 ('Kapitalismuskritik', 0.9078077),
 ('Paternalismus', 0.907747),
 ('Geschlechterrollen', 0.90565866),
 ('Misogynie', 0.9055821),
 ('feministischer', 0.90483505),
 ('Frauenbewegung', 0.9031706),
 ('Sexualmoral', 0.9018312),
 ('Political_Correctness', 0.89458),
 ('Ma

Ergebnis: 

- Word-similarity für "Feminismus" hat nicht besonders gut funktioniert. 
- Auch die Liste der 50 ähnlichsten Wörtern von Word2Vec hat kaum bzw. keinen gleichen Eintrag an der gleichen Stelle, wenn man die Liste 1 zu 1 in Bert übergibt.
--> Dennoch viel bessre Ergebnisse als wenn man Bert von selbst die ähnlichsten Wörter finden lässt.
- Hoffnung macht: Für das Wort "schwul" hat es ganz gut funktioniert. 
--> Vielleicht hier auch nochmal versuchen bzw. vergleichen mit Word2vec
- Für weitere Wörter ausprobieren!

# Versuch mit Roboter

In [ ]:
r = "Roboter"

Roboter = foo(r, tokenizer.vocab)

In [ ]:
Roboter

[('Roboter', 0.99999994),
 ('Europäer', 0.9151234),
 ('Hacker', 0.91160476),
 ('Soldaten', 0.9112953),
 ('Ingenieure', 0.9110926),
 ('Konzerne', 0.9088095),
 ('Erzieher', 0.9074953),
 ('Türken', 0.90698296),
 ('Patienten', 0.90669304),
 ('Forscher', 0.9064501),
 ('Elefanten', 0.90414125),
 ('Rivalen', 0.90064687),
 ('Teenager', 0.9005905),
 ('Teilchen', 0.8999297),
 ('Sprit', 0.89895284),
 ('Mädchen', 0.89863014),
 ('Fachleute', 0.89848375),
 ('Islamisten', 0.89721507),
 ('Mini', 0.8971403),
 ('Experten', 0.89670545),
 ('Geheimdienst', 0.8950706),
 ('Menschen', 0.89493686),
 ('Computer', 0.89422005),
 ('##ungsanlagen', 0.8934453),
 ('Dienste', 0.8933008),
 ('Smartphone', 0.89261764),
 ('Jobs', 0.89241916),
 ('Russen', 0.8916274),
 ('Chinesen', 0.8914742),
 ('Helden', 0.89130765),
 ('Hirn', 0.89044476),
 ('Koffer', 0.889909),
 ('Nutzer', 0.8891503),
 ('Hasen', 0.8888307),
 ('##Technik', 0.8887904),
 ('Kameras', 0.8885826),
 ('Konkurrenten', 0.88834673),
 ('Köpfe', 0.88828814),
 ('Netz',

## Versuch mit den 50 häufigsten Wörtern zu Roboter von Word2Vec

In [ ]:
r = "Roboter"
w = ['Robotern', 'Kirobo', 'Quadrocopter', 'Flugroboter', '3-D-Drucker', '3D-Drucker', 'Roboters', 'ferngesteuerte',
'Oktokopter', 'Kampfroboter', 'Mini-Computer', 'zusammenschrauben', 'computergesteuerten', 'Industrieroboter', 'Pulsmesser',
'Mikrochips', 'durchs_Weltall', 'Dummys', 'Greifarm', 'Wiimote', 'zusammenzubauen', 'Fernsteuerung', 'Roentgenblick', 
'Cyborgs', 'Wetterballon', 'Raumschiffe', 'Quadrokopter', 'Raumanzug', 'Sensor', 'Joystick', 'zusammenbaut',
'Lego-Figuren', 'R2-D2', 'Raumschiff', 'Sensoren', 'Sprachbefehle', 'Astronauten', 'Humanoiden', 'Teleportation',
'3D-Druck', 'Reinraum', 'ferngesteuerten', 'Wearables', 'kuenstlicher_Intelligenz', 'Simulation', 'Wii-Fernbedienung',
'Raumstation', 'Mars_fliegen', 'Clank', 'loeten']

len(w)

50

In [ ]:
Roboter = foo(r, w)

In [ ]:
Roboter

[('Roboters', 0.94991654),
 ('Mini-Computer', 0.9345482),
 ('Robotern', 0.9266374),
 ('Sensoren', 0.9229022),
 ('Mars_fliegen', 0.89751416),
 ('Lego-Figuren', 0.89156234),
 ('3D-Drucker', 0.88843024),
 ('Pulsmesser', 0.8811019),
 ('Wii-Fernbedienung', 0.8666794),
 ('Astronauten', 0.8545989),
 ('durchs_Weltall', 0.85434544),
 ('3D-Druck', 0.85329133),
 ('Greifarm', 0.85271436),
 ('Industrieroboter', 0.8526815),
 ('Simulation', 0.8497277),
 ('ferngesteuerte', 0.8459163),
 ('computergesteuerten', 0.8386327),
 ('Cyborgs', 0.8366375),
 ('Joystick', 0.8312353),
 ('Raumanzug', 0.82696384),
 ('Humanoiden', 0.8215345),
 ('Quadrokopter', 0.81670696),
 ('Sensor', 0.8166772),
 ('Kampfroboter', 0.8144309),
 ('Quadrocopter', 0.8128417),
 ('Reinraum', 0.8079092),
 ('Mikrochips', 0.803485),
 ('Flugroboter', 0.80036193),
 ('3-D-Drucker', 0.79802525),
 ('Roentgenblick', 0.7933546),
 ('Raumschiffe', 0.7925167),
 ('Sprachbefehle', 0.7819722),
 ('Dummys', 0.7790214),
 ('loeten', 0.7760849),
 ('Fernsteuerun

- Bessere Ergebnisse bei word similarity bei Roboter!
- Automatische Ausgabe der ähnlichen Wörter ergibt größtenteils Sinn.
- Wenn man die Liste der 50 ähnlichsten Wörter von Word2Vec an die Bert Funktion übergibt und sich die Liste der 50 ähnlichsten Wörter durch Bert ausgeben lässt, stimmt leider ebenfalls kein Eintrag überein.

# Versuch mit schwadronieren

In [ ]:
r = "schwadronieren"

schw = foo(r, tokenizer.vocab)

In [ ]:
schw

[('##gängen', 0.9420636),
 ('##rächen', 0.9413635),
 ('##werben', 0.93684286),
 ('##schäden', 0.9345042),
 ('##spräsidenten', 0.93436885),
 ('schlugen', 0.9341915),
 ('##willigen', 0.934035),
 ('##regen', 0.93341553),
 ('##risten', 0.9331813),
 ('##ändlern', 0.93292993),
 ('regiert', 0.9326843),
 ('verlieren', 0.9325791),
 ('fürchten', 0.93248904),
 ('stimmen', 0.9321505),
 ('##brechen', 0.9320917),
 ('denkt', 0.93208104),
 ('##renzen', 0.9308175),
 ('##ständnis', 0.93060106),
 ('##arbeiten', 0.93030787),
 ('##gutachten', 0.9298358),
 ('brechen', 0.9297456),
 ('##reifen', 0.92974424),
 ('treibt', 0.9294221),
 ('##bedro', 0.92926604),
 ('##kandidaten', 0.9290831),
 ('##erinnen', 0.92904323),
 ('getrieben', 0.9285644),
 ('##Politik', 0.9283974),
 ('##wertigen', 0.92834),
 ('##geräten', 0.9281259),
 ('##ärmen', 0.92810094),
 ('##gesetzt', 0.92755765),
 ('##sprechen', 0.9274162),
 ('##interessen', 0.92730457),
 ('##Politiker', 0.9271071),
 ('überlegen', 0.92703366),
 ('##begehren', 0.92655

## Versuch mit den 50 häufigsten Wörtern zu schwadronieren von Word2Vec

In [ ]:
r = "schwadronieren"

w = ['schwadroniert', 'mokieren', 'Gequatsche', 'ereifern', 'Frustrierten', 'Gutmenschentum', 'schwadronierte', 
'echauffieren', 'Krokodilstraenen', 'scheinheiligen', 'Herrenwitze', 'Redet', 'Felde_zieht', 'Berliner_Politikbetrieb', 
'Gelaber', 'raesonieren', 'Nase_ruempfen', 'Gutmenschen', 'Geschwafel', 'mokiert', 'witzeln', 'noergeln',
'unangenehme_Wahrheiten', 'menscheln', 'Besserwisserei', 'laestern', 'maulen', 'Rainer_Bruederles', 'Straenge_schlagen',
'unbequeme_Wahrheiten', 'hofieren', 'Wand_malen', 'Blauaeugigkeit', 'doofen', 'Laestern', 'aufzuregen', 'kuschen', 
'Ohren_hauen', 'political_correctness', 'spoettelt', 'herzuziehen', 'Politikbetriebs', 'Noergelei', 'Gejammer',
'Ewiggestrigen', 'mault', 'Tiefstapelei', 'Neiddebatte', 'Merkel-CDU', 'Lamentieren']



In [ ]:
len(w)

50

In [ ]:
schwWord2Vec = foo(r, w)

In [ ]:
schwWord2Vec

[('schwadroniert', 0.9600185),
 ('schwadronierte', 0.9434351),
 ('menscheln', 0.9381891),
 ('echauffieren', 0.93741757),
 ('witzeln', 0.93343794),
 ('hofieren', 0.93053406),
 ('mokiert', 0.9299777),
 ('Lamentieren', 0.9277662),
 ('mokieren', 0.9234423),
 ('Straenge_schlagen', 0.92304665),
 ('Redet', 0.92033994),
 ('spoettelt', 0.9149032),
 ('Besserwisserei', 0.9101396),
 ('unbequeme_Wahrheiten', 0.9077681),
 ('Gutmenschen', 0.9038599),
 ('ereifern', 0.9025406),
 ('kuschen', 0.90214795),
 ('Nase_ruempfen', 0.9020281),
 ('laestern', 0.90164477),
 ('Berliner_Politikbetrieb', 0.8987503),
 ('herzuziehen', 0.8980893),
 ('Tiefstapelei', 0.894892),
 ('unangenehme_Wahrheiten', 0.894744),
 ('aufzuregen', 0.89426357),
 ('raesonieren', 0.89201146),
 ('Frustrierten', 0.8879772),
 ('Wand_malen', 0.8831212),
 ('Neiddebatte', 0.8816809),
 ('mault', 0.8800075),
 ('Gejammer', 0.87931),
 ('Ohren_hauen', 0.877864),
 ('Politikbetriebs', 0.874302),
 ('Ewiggestrigen', 0.8721677),
 ('maulen', 0.86880296),
 ('

Zum ersten Mal bei der Übergabe der Liste der 50 häufigsten Wörter von Word2Vec zu Bert, scheinen die Ergebnisse bei Bert fast besser zu sein als bei Word2Vec:

- Die ersten Wörter passen besser als bei Word2Vec, sowohl vom Wortstamm als auch von der Bedeutung her.
- Bei Word2Vec ist Beispielsweise das Wort Krokodilstränen sehr weit oben, bei Bert weit unten, was richtig ist, da dieses Wort nahezu nichts mit der eigentlichen Bedeutung von schwadronieren zu tun hat.


# Versuch mit Ampel

In [ ]:
z = "Ampel"

amp = foo(z, tokenizer.vocab)

In [ ]:
amp

[('##ungsanlagen', 0.9353636),
 ('##ungsanlage', 0.9326489),
 ('Tempo', 0.9301058),
 ('Tram', 0.92746836),
 ('Netz', 0.927458),
 ('Wind', 0.92599845),
 ('Hitze', 0.9249028),
 ('Strahlen', 0.92461306),
 ('Kran', 0.92391497),
 ('##ioxid', 0.9237995),
 ('##ronomie', 0.92286044),
 ('##Verkehr', 0.9218776),
 ('Feuerwehr', 0.9216151),
 ('##lenkung', 0.92113763),
 ('##strom', 0.9191611),
 ('Verkehr', 0.91901827),
 ('Auto', 0.918748),
 ('Versorg', 0.91867965),
 ('##abgabe', 0.9185366),
 ('Navig', 0.9184116),
 ('##cticut', 0.91748077),
 ('##orne', 0.9170115),
 ('##orten', 0.9163711),
 ('##fahrzeuge', 0.9148034),
 ('##ström', 0.91463584),
 ('Technik', 0.9146014),
 ('##sverkehr', 0.91375),
 ('##überwachung', 0.91317666),
 ('##Länder', 0.9131385),
 ('Mehrheit', 0.912465),
 ('Inland', 0.91243047),
 ('Feuer', 0.9124093),
 ('Wellen', 0.9118715),
 ('##oren', 0.91180545),
 ('Öl', 0.91178674),
 ('##ladung', 0.91055334),
 ('##ränkung', 0.91052717),
 ('Autofahrer', 0.9104778),
 ('##wendung', 0.9103898),
 

## Versuch mit den 50 häufigsten Wörtern zu Ampel von Word2Vec

In [ ]:
z = "Ampel"

a = ['Fussgaengerampel', 'Ampelanlage', 'Ampel_Kreuzung', 'Kreuzung', 'Baustellenampel', 'Ampeln', 'Zebrastreifen',
'Ampelkreuzung', 'Fussgaengerueberweg', 'Linksabbieger', 'Ampel_Rot', 'Bedarfsampel', 'Ueberweg', 'Lichtzeichenanlage',
'Linksabbiegerspur', 'Abbieger', 'Kreisverkehr', 'Geradeausspur', 'Busspur', 'Kreisel', 'Abbiegespur', 'Gruenphase',
'Ampel_rot', 'Bahnuebergang', 'Fussgaengerfurt', 'Rotlicht', 'Linksabbiegespur', 'Stopp-Schild', 'rechts_abbiegen', 
'Sperrflaeche', 'Gruenlicht', 'Querungshilfe', 'Haltelinie', 'Signalanlage', 'Ampelschaltung', 'roten_Ampel',
'Rechtsabbiegerspur', 'Rechtsabbieger', 'Verkehrsinsel', 'Einfaedelspur', 'Mittelinsel', 'Vorfahrtsstrasse',
'Ampel_Gruen', 'geblinkt', 'Fussgaenger', 'Schranke', 'Bahnunterfuehrung', 'Kreuzung_ueberqueren','rote_Ampel', 
'Fahrbahnverengung']

In [ ]:
ampWord2Vec = foo(z, a)
ampWord2Vec

[('Ampelschaltung', 0.97230583),
 ('Ampeln', 0.9641194),
 ('Ampelkreuzung', 0.95996946),
 ('Bedarfsampel', 0.95601165),
 ('Ampelanlage', 0.9457663),
 ('roten_Ampel', 0.9446945),
 ('Ampel_Rot', 0.941439),
 ('rote_Ampel', 0.9335386),
 ('Bahnunterfuehrung', 0.92647684),
 ('Zebrastreifen', 0.92529595),
 ('Rotlicht', 0.9231382),
 ('Ampel_Gruen', 0.9214733),
 ('Signalanlage', 0.91574025),
 ('Schranke', 0.91435754),
 ('Ampel_rot', 0.9126798),
 ('Verkehrsinsel', 0.9092024),
 ('Fahrbahnverengung', 0.9089233),
 ('Einfaedelspur', 0.9072994),
 ('Stopp-Schild', 0.90647227),
 ('Abbiegespur', 0.90450424),
 ('Geradeausspur', 0.903059),
 ('Fussgaengerampel', 0.8982201),
 ('Kreisverkehr', 0.89748573),
 ('Sperrflaeche', 0.89364535),
 ('Bahnuebergang', 0.8919156),
 ('Baustellenampel', 0.8909425),
 ('Linksabbiegespur', 0.8893812),
 ('Kreisel', 0.8848327),
 ('Abbieger', 0.8808465),
 ('Gruenphase', 0.8788023),
 ('Fussgaengerfurt', 0.87735116),
 ('Busspur', 0.87693006),
 ('Haltelinie', 0.87369907),
 ('Rechtsa

Die Übergabe der Liste der 50 häufigsten Wörter von Word2Vec zu Bert, scheinen die Ergebnisse bei Bert fast besser zu sein als bei Word2Vec:
Die ersten Wörter passen besser als bei Word2Vec, sowohl vom Wortstamm als auch von der Bedeutung her.

# Versuch mit Batterie

In [ ]:
h = "Batterie"

batt = foo(h, tokenizer.vocab)

In [ ]:
batt

[('Batterie', 1.0),
 ('diesel', 0.9437215),
 ('Tank', 0.94215125),
 ('Speicher', 0.9375412),
 ('Waffe', 0.9324351),
 ('Elektronik', 0.92920995),
 ('Prototyp', 0.92912495),
 ('Baureihe', 0.9285437),
 ('Erdgas', 0.92427754),
 ('Signal', 0.92409486),
 ('Autob', 0.9212961),
 ('Lade', 0.9212149),
 ('Elektro', 0.92058736),
 ('Marine', 0.92030805),
 ('elektrische', 0.9202407),
 ('Quad', 0.9185043),
 ('Radar', 0.91847897),
 ('Micro', 0.9178629),
 ('Laser', 0.9177788),
 ('Brigade', 0.91726667),
 ('Plattform', 0.9171696),
 ('Stadtbahn', 0.91712314),
 ('Soft', 0.9170824),
 ('Draht', 0.91678494),
 ('basiert', 0.916566),
 ('standard', 0.9164302),
 ('Systems', 0.9162079),
 ('Container', 0.91588473),
 ('Maschine', 0.9158379),
 ('Rail', 0.9157823),
 ('Smart', 0.9157728),
 ('Cab', 0.91567695),
 ('AT', 0.9155847),
 ('Artillerie', 0.9152518),
 ('Solar', 0.91518176),
 ('Flüssigkeit', 0.91430426),
 ('Elektrom', 0.91429126),
 ('Elektriz', 0.9141912),
 ('Turb', 0.9134569),
 ('Modell', 0.91341776),
 ('Dampf',

Interessant: Die Einträge ('Waffe', 0.9324351) und ('Marine', 0.92030805) deuten auf die ältere Bedeutung des Wortes Batterei hin, obwohl hier aktuelle Wikipediaartikel verwendet werden. 
Könnte guter Kandidat für Wortbedeutungswandel sein!

## Versuch mit den 50 häufigsten Wörtern zu Batterie von Word2Vec

In [ ]:
h = "Batterie"

w = ['Akkus', 'Batterien', 'Akku', 'Steckdose', 'Lithium-Ionen-Akkus', 'Lithium-Ionen-Batterie', 'Haushaltssteckdose', 
'Starterbatterie', 'Lichtmaschine', 'Lithium-Ionen-Batterien', 'Bordelektronik', 'Generator', 'Akkumulator', 'Autobatterie', 
'Lithium-Ionen-Akku', 'Traktionsbatterie', 'Rekuperation', 'Lithium-Ionen-Akkumulator', 'Solarmodul', 'Hydraulikpumpe',
'Zuendspule', 'Anlasser', 'Synchrongeneratoren', 'Akkumulatoren', 'Druckspeicher', 'Ladegeraet', 'Verbindungskabel',
'Fahrzeugbatterie', 'Verbrennungsmotor', 'Steuerelektronik', 'Netzteil', 'Energierueckgewinnung','Luftfilter', 'Bremsenergie',
'Kuehlaggregat', 'Motorbetrieb', 'Nebenaggregate', 'Vorschaltgeraet', 'Kraftstoffpumpe', 'Doppelkupplung', 'Nutzbremse', 
'Elektromotors', 'Standheizung', 'Kondensator', 'Stand-by-Modus', 'Ladekabel', 'Zuendverteiler', 'Stromspeicher', 
'Kochplatte', 'Stromgenerator']

In [ ]:
battword2vec = foo(h, w)

In [ ]:
battword2vec

[('Batterien', 0.927508),
 ('Netzteil', 0.91698396),
 ('Lichtmaschine', 0.9168993),
 ('Nutzbremse', 0.91602814),
 ('Druckspeicher', 0.9133895),
 ('Bordelektronik', 0.9130237),
 ('Stromspeicher', 0.91224235),
 ('Elektromotors', 0.91057914),
 ('Solarmodul', 0.9066299),
 ('Energierueckgewinnung', 0.905255),
 ('Kondensator', 0.90514207),
 ('Kuehlaggregat', 0.900311),
 ('Stromgenerator', 0.8996947),
 ('Kochplatte', 0.898462),
 ('Lithium-Ionen-Akkumulator', 0.8975994),
 ('Generator', 0.89543253),
 ('Ladekabel', 0.8945744),
 ('Zuendverteiler', 0.89402425),
 ('Steuerelektronik', 0.8908028),
 ('Vorschaltgeraet', 0.889966),
 ('Steckdose', 0.88665366),
 ('Lithium-Ionen-Akku', 0.886299),
 ('Ladegeraet', 0.8854007),
 ('Verbindungskabel', 0.8838575),
 ('Bremsenergie', 0.88279486),
 ('Lithium-Ionen-Batterie', 0.87755686),
 ('Synchrongeneratoren', 0.8752937),
 ('Rekuperation', 0.87245107),
 ('Lithium-Ionen-Akkus', 0.87188804),
 ('Fahrzeugbatterie', 0.87160957),
 ('Doppelkupplung', 0.8702286),
 ('Lithi

Batterie wird deutlich als elektronischer Speicher erkannt. 

Die Bedeutung dieses Wortes sollte sich ändern, sobald dieses Wort mit Modellen verglichen wird, welche auf älteren Texten trainiert wurden.

# Versuch mit Revolution

In [ ]:
r = "Revolution"

revo = foo(r, tokenizer.vocab)

In [ ]:
revo

[('Revolution', 1.0),
 ('Kommunisten', 0.9243751),
 ('Aufstand', 0.9226436),
 ('Regime', 0.9204639),
 ('Invasion', 0.91828436),
 ('##regierung', 0.91322356),
 ('Ermordung', 0.9117064),
 ('Putsch', 0.9098611),
 ('Front', 0.9089419),
 ('Opposition', 0.90826404),
 ('CIA', 0.90777946),
 ('Eta', 0.9074374),
 ('Stasi', 0.90732425),
 ('demokratische', 0.9072081),
 ('Kommunistischen', 0.9062098),
 ('Libert', 0.9059065),
 ('Bewegung', 0.90556717),
 ('Guer', 0.9053225),
 ('Liberalen', 0.9052406),
 ('RAF', 0.9040671),
 ('Gestapo', 0.9040072),
 ('rechtsextrem', 0.9037759),
 ('revolution', 0.9035758),
 ('Bestrebungen', 0.9025753),
 ('kommunistischen', 0.90164924),
 ('Epoche', 0.9004404),
 ('Pest', 0.9002709),
 ('Verschwörung', 0.90000266),
 ('Erde', 0.8999201),
 ('Islam', 0.89906657),
 ('Reformation', 0.8988886),
 ('Demokratischen', 0.89883405),
 ('Gang', 0.89877796),
 ('Welle', 0.89802694),
 ('Roma', 0.8977653),
 ('Eroberung', 0.8969954),
 ('Viet', 0.8965325),
 ('##regation', 0.8963843),
 ('belage

Ergebnis ist sehr gut für ähnliche Wörter!

## Versuch mit den 50 häufigsten Wörtern zu Revolution von Word2Vec

In [ ]:
r = "Revolution"

q = ['revolutionaeren', 'Rebellion', 'Revolutionen', 'Revolte', 'Aufstand', 'arabischen_Fruehling', 'Sturz_Mubaraks',
'Revolutionaere', 'aegyptischen_Revolution', 'Diktatur', 'revolutionaere', 'Umsturz', 'Arabischen_Fruehling',
'Mubarak-Regime', 'Arabellion', 'Konterrevolution', 'Arabischen_Fruehlings', 'Demokratie', 'proletarische_Revolution',
'Putsch', 'Militaerherrschaft', 'Sozialismus', 'Ayatollahs', 'Staatsstreich', 'arabischen_Fruehlings',
'arabische_Fruehling', 'Sturz_Hosni', 'Massenprotest', 'Umwaelzung', 'Ben_Ali', 'Mursi-Sturz', 'Aufstaende',
'Machthaber_Husni', 'Aufstands', 'Sturz_Diktators', 'Ben_Alis', 'aegyptische_Volk', 'Demokratiebewegung',
'Kapitalismus', 'Sturz_Regimes', 'Militaerputsch', 'Volksaufstand', 'Plutokratie', 'Tamarud', 'Volksbewegung',
'Sozialismus_Kommunismus', 'Aufstandes', 'Absetzung_Mursis', 'Absetzung_Praesidenten', 'Strassenproteste']



In [ ]:
len(q)

50

In [ ]:
revoword2vec = foo(r, q)

In [ ]:
revoword2vec

[('proletarische_Revolution', 0.95322317),
 ('aegyptischen_Revolution', 0.9487009),
 ('Demokratiebewegung', 0.93713677),
 ('Revolutionen', 0.9303679),
 ('Aufstand', 0.9226436),
 ('Aufstandes', 0.91707146),
 ('Aufstands', 0.91483843),
 ('Revolte', 0.9114862),
 ('Putsch', 0.9098611),
 ('Umsturz', 0.9080151),
 ('Volksbewegung', 0.90390694),
 ('Plutokratie', 0.8984257),
 ('Rebellion', 0.89810175),
 ('Diktatur', 0.89260304),
 ('Sozialismus', 0.89248365),
 ('Kapitalismus', 0.89179134),
 ('Sozialismus_Kommunismus', 0.88963395),
 ('Militaerherrschaft', 0.88481545),
 ('Konterrevolution', 0.8818501),
 ('Volksaufstand', 0.8813724),
 ('Demokratie', 0.87760705),
 ('Absetzung_Praesidenten', 0.8775046),
 ('Revolutionaere', 0.8740771),
 ('Sturz_Regimes', 0.87389606),
 ('Ayatollahs', 0.8721328),
 ('Strassenproteste', 0.8716415),
 ('aegyptische_Volk', 0.8705824),
 ('Arabischen_Fruehlings', 0.86577797),
 ('Arabellion', 0.8543676),
 ('revolutionaere', 0.84956723),
 ('Sturz_Diktators', 0.8456704),
 ('arabi

Auch im Listenvergleich zwischen Bert und Word2Vec hat Bert ein besseres Ergebnis. 

# Versuch mit krass

In [24]:
k = "krass"

kra = foo(k, tokenizer.vocab)

In [25]:
kra

[('##lässlich', 0.9593392),
 ('echt', 0.9588985),
 ('##echt', 0.9555639),
 ('##öglich', 0.9551055),
 ('##ursache', 0.9524038),
 ('##nehmlich', 0.9511525),
 ('##zimmer', 0.95066446),
 ('##denklich', 0.95066357),
 ('##lässig', 0.9492998),
 ('##lässigkeit', 0.94909424),
 ('##reffend', 0.94871134),
 ('ordentlich', 0.9482086),
 ('##eutsch', 0.94817734),
 ('##klausel', 0.9474717),
 ('##spieler', 0.9457723),
 ('##kau', 0.945663),
 ('absolut', 0.94517666),
 ('##kom', 0.9449258),
 ('##nick', 0.94472426),
 ('##ötzlich', 0.94401026),
 ('##läufig', 0.94366896),
 ('##ätzlich', 0.9434402),
 ('Tatsächlich', 0.9433067),
 ('##parade', 0.9430621),
 ('##napp', 0.94264436),
 ('##schön', 0.94246054),
 ('##änglich', 0.94201916),
 ('##schwanz', 0.94165367),
 ('##lässige', 0.9414717),
 ('##anspruch', 0.94138056),
 ('##klasse', 0.9410073),
 ('richtig', 0.94066995),
 ('##key', 0.94053197),
 ('##ahnung', 0.9405142),
 ('##läss', 0.9401874),
 ('##träge', 0.9397464),
 ('##merkmal', 0.9396783),
 ('##lug', 0.9395783)

## Versuch mit den 50 häufigsten Wörtern zu krass von Word2Vec

In [27]:
k = "krass"

In [29]:
u = ['bloed', 'geil', 'bescheuert', 'furchtbar', 'deprimierend', 'eklig', 'doof', 'schlimm', 'komisch', 'nervig', 
'peinlich', 'daemlich', 'schrecklich', 'cool', 'fies', 'scheisse', 'echt', 'irre', 'unsexy', 'laecherlich', 
'irgendwie_komisch', 'pervers', 'albern', 'deppert', 'abartig', 'grotesk', 'unterste_Schublade', 'schoen', 'toll', 
'beaengstigend', 'unverschaemt', 'banal', 'witzig', 'spassig', 'lustig', 'ulkig', 'kribbelig', 'putzig', 'ekelig', 
'bescheuerte', 'kleinkariert', 'Laecherlich', 'erschreckend', 'absolute_Frechheit', 'pubertaer', 'aufregend', 
'wehgetan', 'soooo', 'ekelhaft', 'fuerchterlich']

In [30]:
len(u)

50

In [31]:
kra = foo(k, u)

In [32]:
kra

[('ekelig', 0.9602368),
 ('echt', 0.9588985),
 ('albern', 0.9559725),
 ('ekelhaft', 0.95062405),
 ('laecherlich', 0.9438229),
 ('komisch', 0.9425248),
 ('banal', 0.94235694),
 ('eklig', 0.9416559),
 ('irgendwie_komisch', 0.9391117),
 ('fuerchterlich', 0.9375425),
 ('doof', 0.9374372),
 ('cool', 0.9366845),
 ('kribbelig', 0.93570024),
 ('lustig', 0.9344115),
 ('nervig', 0.9332036),
 ('peinlich', 0.9331346),
 ('fies', 0.9324467),
 ('putzig', 0.93155366),
 ('Laecherlich', 0.93019813),
 ('absolute_Frechheit', 0.92974997),
 ('furchtbar', 0.9270215),
 ('witzig', 0.92409),
 ('schrecklich', 0.9233545),
 ('schlimm', 0.91961217),
 ('unsexy', 0.91865623),
 ('pervers', 0.91559196),
 ('deppert', 0.90801376),
 ('irre', 0.9051044),
 ('scheisse', 0.9012579),
 ('toll', 0.90043646),
 ('soooo', 0.8960107),
 ('aufregend', 0.891508),
 ('erschreckend', 0.89058393),
 ('wehgetan', 0.8891205),
 ('deprimierend', 0.8824581),
 ('daemlich', 0.8747699),
 ('beaengstigend', 0.87300336),
 ('kleinkariert', 0.86961764),

Das Ergebnis für die Wortähnlichkeit mit Bert war nicht gut. War auch zu erwarten, vielleicht Wort zu kurz? Bei längeren Wörtern wie Revolution oder Batterie waren die Ergebnisse viel besser. Vielleicht weil Nomen? Im Vergleich der Wortähnlichkeitslisten ergaben sich kaum erwähnenswerte Ergebnisse, kein Eintrag der Listen stimmt genau überein.